<a href="https://colab.research.google.com/github/TomoharuKurosu/Tomoharu_DS2/blob/main/%E7%A4%BE%E4%BC%9A%E7%B5%8C%E6%B8%88%E6%83%85%E5%A0%B1_%E8%A7%A3%E8%AA%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#社会経済情報_解説
# %% init
from setting import *
# %% ufd
def key_clnser(df):
    return df.rename(columns={
        "宛名コード":"宛名番号",
        "住民コード":"宛名番号",
        "住民番号":"宛名番号",
        "個人番号":"宛名番号",
    })
def to_onehot(df):
    return (
        pd
        .get_dummies(df.astype(str).replace('nan',np.nan), drop_first=False)
        .rename(columns=lambda c: "_".join(c.split("_")[:-1]) + "=" + c.split("_")[-1] + "_フラグ")
        * 1
    )
def shape(df):
    print(df.shape)
    return df
def cumsum(df, key="世帯番号"):
    df = df.sort_values([key, "年度"])
    return (
        df
        [[key, "年度"]]#同じ "世帯番号" に属する行はすべて連続しており、その中で年度順に並びます。
        .join(
            df
            .drop(columns="年度")
            .groupby(key)
            .cumsum()
        )
    )
def ffill(df, key="世帯番号"):
    df = df.sort_values([key, "年度"])
    return (
        df
        [[key, "年度"]]
        .join(
            df
            .drop(columns="年度")
            .groupby(key)
            .fillna(method="ffill")
            .fillna(0)
        )
    )
# %%
p_in = "../data_clns+/0.母集団.csv"
df_pop = pd.read_csv(p_in)
df_pop
# %%
setai_map = (
    pd
    .read_csv("../data_raw/市民室_1ファイル/001_住基データ.csv")
    [["宛名番号", "世帯番号"]]
    .drop_duplicates()
    .pipe(check_df)
)
# %%
p_in = "../data_raw/生活福祉課_1ファイル/被保護世帯員個人情報一覧.csv"
p_out = "../data_clns+/3.生活保護.csv"
_df = (
    pd
    .read_csv(p_in)
    .merge(
        setai_map
        , on="宛名番号"
        , how="inner"
    )
    .assign(年度 = lambda df: pd.to_datetime(df["個人開始年月日"]).dt.year)
    .sort_values("年度")
    .groupby("宛名番号")
    .nth(0)
    .groupby(["世帯番号", "年度"], as_index=False)
    ["宛名番号"].nunique()
    .rename(columns={"宛名番号":"被保護世帯員個人情報一覧_世帯被生保延べ人数"})
    .pipe(cumsum)
    .merge(
        df_pop
        [["世帯番号", "年度"]]
        .drop_duplicates()
        .sort_values(["世帯番号", "年度"])
        , on=["世帯番号", "年度"]
        , how="outer"
    )
    .pipe(ffill)
)
_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
p_in = "../data_raw/子育て支援室_6ファイル/児童手当受給者_受給者一覧.csv"
p_out = "../data_clns+/3.児手.csv"
_df = (
    pd
    .read_csv(p_in)
    [["宛名番号", "世帯番号", "認定日"]]
    # .columns
    .assign(年度 = lambda df: pd.to_datetime(df["認定日"]).dt.year)
    .sort_values("年度")
    .groupby("宛名番号")
    .nth(0)
    .groupby(["世帯番号", "年度"], as_index=False)
    ["宛名番号"].nunique()
    .rename(columns={"宛名番号":"児童手当受給者ー受給者一覧_世帯被児手延べ人数"})
    .pipe(cumsum)
    .merge(
        df_pop
        [["世帯番号", "年度"]]
        .drop_duplicates()
        .sort_values(["世帯番号", "年度"])
        , on=["世帯番号", "年度"]
        , how="outer"
    )
    .pipe(ffill)
)
_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
p_in = "../data_raw/子育て支援室_6ファイル/児童扶養手当児童索引簿.csv"
p_out = "../data_clns+/3.児扶.csv"
_df = (
    pd
    .read_csv(p_in)
    .pipe(key_clnser)
    .drop(columns="宛名番号")
    .rename(columns={"児童宛名コード":"宛名番号"})
    [["宛名番号", "認定年月日"]]
    .pipe(shape)
    .assign(年度 = lambda df:
        pd
        .to_datetime(
            df["認定年月日"]
            .apply(lambda x:
                str(int(x[1:3])+1988)
                + x.split(".")[1].strip().zfill(2)
                + x.split(".")[2].strip().zfill(2)
            )#認定年月日 を使用して年度 (年度) を計算し、新しいカラムとして追加します。具体的には、認定年月日 の年部分を 1988 年を基準に補正し、日付形式に変換してから 年 のみを抽出しています。
        )
        .dt.year
    )
    .drop(columns="認定年月日")
    .sort_values("年度")
    .groupby("宛名番号")
    .nth(0)#宛名番号 でグループ化し、各グループの最初の行を選択します。
    .assign(児童扶養手当児童索引簿_児扶受給フラグ = 1)
    # .pipe(cumsum)
    .merge(
        df_pop
        [["宛名番号", "年度"]]
        .drop_duplicates()
        .sort_values(["宛名番号", "年度"])
        , on=["宛名番号", "年度"]
        , how="outer"
    )
    .pipe(
        ffill
        , key="宛名番号"
    )#ffill 関数を使って、宛名番号 に基づいて前の行の値で欠損値を補完します。
)
_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
p_in = "../data_raw/障がい福祉課_7ファイル/特別児童扶養手当.csv"
p_out = "../data_clns+/3.特別児扶.csv"

_df = (
    pd
    .read_csv(p_in)
    # .pipe(key_clnser)
    .rename(columns={"（児童）宛名番号":"宛名番号"})
    [["宛名番号", "認定日付"]]
    .pipe(shape)
    .dropna()
    .assign(年度 = lambda df: pd.to_datetime(df["認定日付"]).dt.year)
    .drop(columns="認定日付")
    .sort_values("年度")
    .groupby("宛名番号")
    .nth(0)
    .assign(特別児童扶養手当_特別児扶受給フラグ = 1)
    .merge(
        df_pop
        [["宛名番号", "年度"]]
        .drop_duplicates()
        .sort_values(["宛名番号", "年度"])
        , on=["宛名番号", "年度"]
        , how="outer"
    )
    .pipe(
        ffill
        , key="宛名番号"
    )
)
_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
p_in = "../data_raw/子育て支援室_6ファイル/ひとり親医療受給者.csv"
p_out = "../data_clns+/3.ひとり親医療受給者.csv"
_df = (
    pd
    .read_csv(p_in)
    .rename(columns=lambda c: c.replace("［基本情報］", ""))
    [['世帯番号', '資格取得日']]
    .pipe(shape)
    .dropna()
    .assign(年度 = lambda df: pd.to_datetime(df["資格取得日"].astype(str)).dt.year)
    .drop(columns="資格取得日")
    .sort_values("年度")
    .assign(ひとり親医療受給者_受給回数 = 1)
    .pipe(cumsum)
    .merge(
        df_pop
        [["世帯番号", "年度"]]
        .drop_duplicates()
        .sort_values(["世帯番号", "年度"])
        , on=["世帯番号", "年度"]
        , how="outer"
    )
    .pipe(
        ffill
        , key="世帯番号"
    )
)
_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
p_in = "../data_raw/子育て支援室_6ファイル/こども医療受給者.csv"
p_out = "../data_clns+/3.こども医療受給者.csv"
_df = (
    pd
    .read_csv(p_in)
    .rename(columns=lambda c: c.replace("［基本情報］", ""))
    [['世帯番号', '資格取得日']]
    .pipe(shape)
    .dropna()
    .assign(年度 = lambda df: pd.to_datetime(df["資格取得日"].astype(str)).dt.year)
    .drop(columns="資格取得日")
    .sort_values("年度")
    .assign(こども医療受給者_受給回数 = 1)
    .pipe(cumsum)
    .merge(
        df_pop
        [["世帯番号", "年度"]]
        .drop_duplicates()
        .sort_values(["世帯番号", "年度"])
        , on=["世帯番号", "年度"]
        , how="outer"
    )
    .pipe(
        ffill
        , key="世帯番号"
    )
)

_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
fname = "自立支援医療_厚生医療"
p_in = f"../data_raw/障がい福祉課_7ファイル/{fname}.csv"
p_out = f"../data_clns+/3.{fname}.csv"
_df = (
    pd
    .read_csv(p_in)
    [[
        # '個人番号',
        '世帯番号',
        '受給開始日',
    ]]
    .pipe(key_clnser)
    .dropna()
    .assign(年度 = lambda df: pd.to_datetime(df["受給開始日"].astype(str)).dt.year)
    .drop(columns="受給開始日")
    .sort_values("年度")
    .assign(**{f"{fname}_受給回数" : 1})
    .pipe(cumsum)
    .merge(
        df_pop
        [["世帯番号", "年度"]]
        .drop_duplicates()
        .sort_values(["世帯番号", "年度"])
        , on=["世帯番号", "年度"]
        , how="outer"
    )
    .pipe(
        ffill
        , key="世帯番号"
    )
    # .describe()
)

_df.to_csv(p_out, index=False)
pd.read_csv(p_out)
# %%
fname = "自立支援医療_精神通院"
p_in = f"../data_raw/障がい福祉課_7ファイル/{fname}.csv"
p_out = f"../data_clns+/3.{fname}.csv"

(
    pd
    .read_csv(p_in)
    # .columns
    [[
        # '個人番号',
        '世帯番号',
        '有効開始日付',
    ]]
    .dropna()
    .assign(年度 = lambda df: pd.to_datetime(df["有効開始日付"].astype(str)).dt.year)
    .drop(columns="有効開始日付")
    .sort_values("年度")
    .assign(**{f"{fname}_受給回数" : 1})
    .pipe(cumsum)
    .merge(
        df_pop
        [["世帯番号", "年度"]]
        .drop_duplicates()
        .sort_values(["世帯番号", "年度"])
        , on=["世帯番号", "年度"]
        , how="outer"
    )
    .pipe(
        ffill
        , key="世帯番号"
    )
)

_df.to_csv(p_out, index=False)
pd.read_csv(p_out)
# %%
fname = "自立支援医療_育成医療"
p_in = f"../data_raw/障がい福祉課_7ファイル/{fname}.csv"
p_out = f"../data_clns+/3.{fname}.csv"

(
    pd
    .read_csv(p_in)
    [[
        # '個人番号',
        '世帯番号',
        '受給開始日',
    ]]
    .pipe(key_clnser)
    .dropna()
    .assign(年度 = lambda df: pd.to_datetime(df["受給開始日"].astype(str)).dt.year)
    .drop(columns="受給開始日")
    .sort_values("年度")
    .assign(**{f"{fname}_受給回数" : 1})
    .pipe(cumsum)
    .merge(
        df_pop
        [["世帯番号", "年度"]]
        .drop_duplicates()
        .sort_values(["世帯番号", "年度"])
        , on=["世帯番号", "年度"]
        , how="outer"
    )
    .pipe(
        ffill
        , key="世帯番号"
    )
)

_df.to_csv(p_out, index=False)
pd.read_csv(p_out)
# %%
"../data_raw/障がい福祉課_7ファイル/"